$$
\boldsymbol{x} \in \mathbb{R}^{K \times 1} \quad \text{Portfolio’s active factor exposures (e.g., DTS relative to benchmark)}
$$

$$
\boldsymbol{b}_{\text{raw}} \in \mathbb{R}^{K \times 1} \quad \text{Benchmark’s factor exposures (DTS across sectors)}
$$

$$
\boldsymbol{b} \in \mathbb{R}^{K \times 1} \quad \text{Normalized benchmark exposures (defines credit beta direction)}
$$

$$
\Sigma \in \mathbb{R}^{K \times K} \quad \text{Factor return covariance matrix}
$$

$$
\boldsymbol{x}_{\parallel} \in \mathbb{R}^{K \times 1} \quad \text{Projection of portfolio exposure onto credit beta direction}
$$

$$
\boldsymbol{x}_{\perp} \in \mathbb{R}^{K \times 1} \quad \text{Residual exposures orthogonal to credit beta}
$$

$$
\sigma_{\text{beta}} \quad \text{Volatility contribution from credit beta exposure}
$$

$$
\sigma_{\text{resid}} \quad \text{Volatility contribution from residual sector/style exposure}
$$

$$
\sigma_{\text{xs}} \quad \text{Total expected credit tracking error}
$$

$$
\boldsymbol{b} = \frac{\boldsymbol{b}_{\text{raw}}}{\| \boldsymbol{b}_{\text{raw}} \|}
$$

$$
\boldsymbol{x}_{\parallel} = (\boldsymbol{b}^\top \boldsymbol{x}) \cdot \boldsymbol{b}
$$

$$
\boldsymbol{x}_{\perp} = \boldsymbol{x} - \boldsymbol{x}_{\parallel}
$$

$$
\sigma_{\text{beta}} = \sqrt{ \boldsymbol{x}_{\parallel}^\top \Sigma \boldsymbol{x}_{\parallel} }
$$

$$
\sigma_{\text{resid}} = \sqrt{ \boldsymbol{x}_{\perp}^\top \Sigma \boldsymbol{x}_{\perp} }
$$

$$
\sigma_{\text{xs}} = \sqrt{ \boldsymbol{x}^\top \Sigma \boldsymbol{x} } = \sqrt{ \sigma_{\text{beta}}^2 + \sigma_{\text{resid}}^2 }
$$


In [ ]:
import numpy as np
import logging

def decompose_credit_te(Sigma, rel_exp, bench_exp, verbose=False):
    """
    Decompose tracking error (TE) into credit beta and residual components.

    Parameters
    ----------
    Sigma : np.ndarray, shape (K, K)
        Factor return covariance matrix
    rel_exp : np.ndarray, shape (K,) or (K, 1)
        Active factor exposure (portfolio - benchmark)
    bench_exp : np.ndarray, shape (K,) or (K, 1)
        Benchmark factor exposures (DTS per sector, unnormalized)
    verbose : bool
        If True, logs all intermediate calculations in detail

    Returns
    -------
    dict
        {
            'total_te': float,
            'beta_te': float,
            'residual_te': float,
            'beta_fraction': float,
            'residual_fraction': float
        }
    """

    def log(msg, *args):
        if verbose:
            print("[TE Decomp] " + msg.format(*args))

    # Ensure column vectors
    x = rel_exp.reshape(-1, 1)
    b_raw = bench_exp.reshape(-1, 1)

    log("Input rel_exp shape: {}, bench_exp shape: {}", x.shape, b_raw.shape)

    # Normalize benchmark exposure to get unit beta direction
    norm_b = np.linalg.norm(b_raw)
    if norm_b == 0:
        raise ValueError("Benchmark exposure vector is zero — cannot normalize.")
    b = b_raw / norm_b

    log("Benchmark norm: {:.6f}", norm_b)
    log("First 5 elements of normalized benchmark b:\n{}", b[:5].flatten())

    # Projection of x onto b
    projection_scalar = float(b.T @ x)
    x_parallel = projection_scalar * b
    x_residual = x - x_parallel

    log("Projection scalar (x · b): {:.6f}", projection_scalar)
    log("First 5 elements of x_parallel:\n{}", x_parallel[:5].flatten())
    log("First 5 elements of x_residual:\n{}", x_residual[:5].flatten())

    # Compute variance terms
    beta_var = float(x_parallel.T @ Sigma @ x_parallel)
    resid_var = float(x_residual.T @ Sigma @ x_residual)
    total_var = float(x.T @ Sigma @ x)

    log("Variance(beta): {:.10f}", beta_var)
    log("Variance(resid): {:.10f}", resid_var)
    log("Variance(total): {:.10f}", total_var)
    log("Sum of components: {:.10f}", beta_var + resid_var)

    # Check numerical consistency
    if not np.isclose(beta_var + resid_var, total_var, rtol=1e-5):
        log("⚠️ WARNING: Variance components do not sum to total variance. Check projection math.")

    # Convert to TE
    beta_te = np.sqrt(beta_var)
    resid_te = np.sqrt(resid_var)
    total_te = np.sqrt(total_var)

    return {
        "total_te": total_te,
        "beta_te": beta_te,
        "residual_te": resid_te,
        "beta_fraction": beta_var / total_var if total_var > 0 else 0.0,
        "residual_fraction": resid_var / total_var if total_var > 0 else 0.0,
    }


In [3]:
import numpy as np

def pca_variance_explained(cov_matrix: np.ndarray, weights: np.ndarray = None):
    """
    Perform PCA on a covariance matrix and return the (optionally weighted) variance
    explained by the first principal component (PC1).

    Parameters
    ----------
    cov_matrix : np.ndarray of shape (K, K)
        Covariance matrix of K factors
    weights : np.ndarray of shape (K,) or (K, 1), optional
        Optional vector of benchmark factor weights (e.g., DTS).
        If provided, computes the percent of benchmark risk explained by PC1.

    Returns
    -------
    dict
        {
            'explained_variance_pc1': float,  # unweighted ratio
            'explained_weighted_risk_pc1': float or None,  # only if weights provided
            'eigenvalues': np.ndarray,
            'eigenvectors': np.ndarray,
        }
    """
    # Eigen decomposition
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
    idx = np.argsort(eigenvals)[::-1]
    eigenvals_sorted = eigenvals[idx]
    eigenvecs_sorted = eigenvecs[:, idx]

    # Unweighted total variance
    total_var = np.sum(eigenvals_sorted)
    explained_var_pc1 = eigenvals_sorted[0] / total_var

    result = {
        'explained_variance_pc1': explained_var_pc1,
        'explained_weighted_risk_pc1': None,
        'eigenvalues': eigenvals_sorted,
        'eigenvectors': eigenvecs_sorted
    }

    if weights is not None:
        # Ensure weights are 1D column vector
        weights = weights.reshape(-1, 1)
        total_risk = float(weights.T @ cov_matrix @ weights)

        # Risk from PC1 projection
        pc1 = eigenvecs_sorted[:, 0].reshape(-1, 1)
        pc1_proj = float(weights.T @ pc1 @ pc1.T @ cov_matrix @ pc1 @ pc1.T @ weights)

        # Alternatively, just:
        # pc1_proj = float((weights.T @ pc1)**2 * eigenvals_sorted[0])

        result['explained_weighted_risk_pc1'] = pc1_proj / total_risk

    return result
